Configure OpenAI API Key

In [ ]:
# Install the LangChain python package
%pip install --upgrade --quiet langchain langchain-openai

!pip install langchain_community -qqq
!pip install langchain-experimental -qqq
!pip install -qU langchain-anthropic


In [ ]:
from langchain_anthropic import ChatAnthropic
# Initializing Anthropic LLM model
chat = ChatAnthropic(model="claude-3-5-sonnet-20241022", api_key="sk-ant-api03-")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

In [ ]:
import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

Classify emails

In [ ]:
class Classification(BaseModel):
    """ Model class for defining the structured output for the LLM"""
    category: str = Field(description="Either product inquiry or order request")

classifier = chat.with_structured_output(Classification)

In [ ]:

prompt = """ Using the email body and subject provided, classify the message into either Product inquiry or
    Order request.

    """
category_list = []
# iterating through the email list
for index, row in emails_df.iterrows():
  output_dict = {'email_id': row['email_id']}
  # concatenating the prompt message with the email message
  message = prompt + "subject:" + str(row["subject"]) + "message:" + str(row["message"])

  # using the LLM to classify the email messages
  output = classifier.invoke(message)
  output_dict['category'] = output.category
  category_list.append(output_dict)

print(category_list)

In [ ]:
# save the generated data in csv format
df_category = pd.DataFrame(category_list)
df_category.to_excel('classification.xlsx', index=False)

 Process order requests

In [ ]:
%%capture --no-stderr
%pip install --upgrade --quiet langgraph

In [ ]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

# change the data frame into a SQL database
# for easy access and manipulation for the LLM
engine = create_engine("sqlite:///products_catalog.db")
products_df.to_sql("products_catalog", engine, if_exists="replace")
db = SQLDatabase(engine=engine)
db.run("SELECT * FROM products_catalog LIMIT 2")

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

# tools to be used by the LLM agent to access the database

toolkit = SQLDatabaseToolkit(db=db, llm=chat)
tools = toolkit.get_tools()
tools

In [ ]:
from langchain_core.messages import SystemMessage
SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements other than UPDATE stock levels to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

processing_prompt = """
    Using the email body and subject provided, for each of the order request
    verify product availability in stock. If the order can be fulfilled, 'created'.
    If the order cannot be fulfilled due to insufficient stock, 'out of stock'.
    Also generate professional response emails based on the order processing results:
If the order is fully processed, inform the customer and provide product details.
If the order cannot be fulfilled or is only partially fulfilled, explain the situation, specify the out-of-stock items, and suggest alternatives or options (e.g., waiting for restock).
Also include the quantity demanded in the output and update the stock level of the product """

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent


class OrderProcessor(BaseModel):
    """ The basemodel class for getting
        a structured output"""
    product_id: str = Field(description="The product_id included in the mail")
    quantity: int = Field(description="The quantity of the product included in the mail")
    status: str = Field(description="Order status", enum=["created", "out of stock"])
    response: str = Field(description="Professional response mail")

# creating a react agent that uses chat model, tools, and system message
agent_executor = create_react_agent(chat, tools, messages_modifier=system_message)

In [ ]:
# Separate order request email_id from product inquiry email_id
ordered_request = df_category[df_category['category'] == 'order request']
order_status = []
order_response = []
output = None

# iterate through order request email only
for index, row in ordered_request.iterrows():
  # getting the email from the email data frame
  email = emails_df[emails_df['email_id'] == row['email_id']]
  message = processing_prompt + "subject:" + str(email["subject"].values[0]) + '\n' + "message:" + str(email["message"].values[0])
  # using the prompt and email message on the react agent
  agent_output = agent_executor.invoke({'messages':message})

  # using the output from the agent to get a structured output
  structured_response = chat.with_structured_output(OrderProcessor)
  output = structured_response.invoke(agent_output['messages'][-1].content)

  # converting the structured output into a list of dictionaries
  order_dict = {'email ID': row['email_id'], 'product ID': output.product_id, 'quantity': output.quantity, 'status': output.status}
  order_status.append(order_dict)

  response_dict = {'email ID': row['email_id'], 'response': output.response}
  order_response.append(response_dict)

print(order_status)
print(order_response)

In [ ]:
db.run("SELECT * FROM products_catalog WHERE product_id='LTH0976'")

In [ ]:
# converting the list of dictionaries into a data frame
df_reponse = pd.DataFrame(order_response)
df_status = pd.DataFrame(order_status)

# converting the data frame to csv files
df_reponse.to_excel('order-response.xlsx', index=False)
df_status.to_excel('order-status.xlsx', index=False)

 Handle product inquiry

In [ ]:
from langchain_community.agent_toolkits import create_sql_agent
class InquiryProcessor(BaseModel):
    """ Model class for products inquiry structured output"""
    response: str = Field(description="Response")


inquiry_prompt = """
    Generate a professional and concise response to product inquiries using relevant
    information from the product catalog.
    """
# SQL agent
agent_executor = create_sql_agent(chat, db=db, verbose=False)

In [ ]:
# Separate order request email_id from product inquiry email_id

product_inquiry = df_category[df_category['category'] == 'product inquiry']
output_inquiry_list = []

# iterate through product inquiry email only
for index, row in product_inquiry.iterrows():
  # getting the email from the email data frame
  email = emails_df[emails_df['email_id'] == row['email_id']]
  mail =  f'subject: {email["subject"].values[0]}\n message: {email["message"].values[0]}'
  input = inquiry_prompt + mail
  #using the prompt and email message on the sql agent
  output = agent_executor.invoke({'input': input})
  response_data = {'email ID': row['email_id'], 'response': output['output']}
  output_inquiry_list.append(response_data)
  break

print(output_inquiry_list)

In [ ]:
# converting the list of dictionaries into a data frame
df_inquiry = pd.DataFrame(output_inquiry_list)

# converting the data frame to csv files
df_inquiry.to_excel('inquiry-response.xlsx', index=False)